In [53]:
# import the neccessary libs
from matplotlib.patches import ConnectionPatch
import matplotlib.pyplot as plt
import numpy as np 
import scipy.spatial.distance as dist

* Speech is a MultiVariate

In [54]:
from scipy.io import wavfile
import IPython.display as ipyd
import librosa
import librosa.display

In [55]:
# load the required speech sample file 
hello1 = "../Lecture/audio/hello1.wav"
hello2 = "../Lecture/audio/hello2.wav"
goodbye = "../Lecture/audio/goodbye.wav"

In [56]:
sampling_freq_1, metadata_1 = wavfile.read(hello1) # the metadata is an audio time series
ipyd.Audio(rate=sampling_freq_1, data=metadata_1)

In [57]:
sampling_freq_2, metadata_2 = wavfile.read(hello2)
ipyd.Audio(rate=sampling_freq_2, data=metadata_2)

* with hello1 and hello2, let us mfcc them and find the alignment cost

In [58]:
# Mel Scale Spectogram of hello1
n_fft_1 = int(0.025*sampling_freq_1)
hop_length_1 = int(0.01*sampling_freq_1)

n_fft_2 = int(0.025*sampling_freq_2)
hop_length_2 = int(0.01*sampling_freq_2)

In [59]:
mel_spec_1 = librosa.feature.melspectrogram(
    
    y=metadata_1/1.0, #audio time series
    sr=sampling_freq_1, 
    n_mels=40,
    n_fft=n_fft_1,
    hop_length=hop_length_1
)
log_mel_spec_1 = np.log(mel_spec_1)

In [60]:
# # plot the log-mel spectogram 
# fig, ax = plt.subplots(figsize=(7,5))
# ax.imshow(log_mel_spec_1, origin="lower", interpolation="nearest")
# plt.ylabel('Feature Dimension')
# plt.xlabel("Frames")

In [61]:
# Mel Scale Spectogram of hello2
mel_spec_2 = librosa.feature.melspectrogram(

    y=metadata_2/1.0,
    sr=sampling_freq_2,
    n_mels=40,
    n_fft=n_fft_2,
    hop_length=hop_length_2
)

log_mel_spec_2 = np.log(mel_spec_2)

In [62]:
# # plot out spectogram

# fig, ax = plt.subplots(figsize=(7,5))
# ax.imshow(log_mel_spec_2, origin='lower', interpolation='nearest')
# plt.ylabel("Feature Dimension")
# plt.xlabel("Frames")

`calculate the cost of hello1 and hello2`

In [63]:
hello1_signal = log_mel_spec_1.T
hello2_signal = log_mel_spec_2.T

In [64]:
# Define the DTW function
def dp(dist_matrix):
    
    N, M = dist_matrix.shape

    # extract dimensionality -> Initialize the cost matrix
    cost_matrix = np.zeros((N+1, M+1))
    for i in range(1, N+1):
        cost_matrix[i,0] = np.inf 
    for i in range(1, M+1):
        cost_matrix[0,i] = np.inf 


    # Flll the cost matrix while keepng the traceback information
    traceback_matrix = np.zeros((N, M))
    for i in range(N):
        for j in range(M):
            penalty = [
                cost_matrix[i, j], # match(0)
                cost_matrix[i, j+1], # insertion(1)
                cost_matrix[i+1, j] # deletion(2)
            ]

            i_penalty = np.argmin(penalty)

            cost_matrix[i+1, j+1] = dist_matrix[i,j]+penalty[i_penalty]
            traceback_matrix[i,j] = i_penalty

    # Let our traceback be from bottom right.
    i = N-1
    j = M-1
    path = [(i,j)]

    while i > 0 or j > 0:
        tb_type = traceback_matrix[i,j]
        if tb_type == 0:
            # match
            i = i-1
            j = j-1
        elif tb_type == 1:
            # insertion
            i = i -1 
        elif tb_type == 2:
            # deletion
            j = j -1 
        path.append((i,j))

    # strip infinity edges before returning
    cost_matrix = cost_matrix[1:, 1:]
    return (path[::-1], cost_matrix)


In [65]:
dist_matrix = dist.cdist(hello1_signal, hello2_signal, 'cosine')
path, cost_matrix = dp(dist_matrix)
Alignment_cost  = cost_matrix[-1,-1]
Alignment_cost

0.602585818307543